In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import random
import numba
import statsmodels.api as sm
import pandas as pd
from scipy.stats import pearsonr
import os
import sys
import shapely.geometry as sgeom
from shapely.prepared import prep
from shapely.geometry import box
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import cartopy.feature as cfeature
from shapely.ops import unary_union
# Step 1: Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "/scratch/ivyglade/pdo/"))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Step 2: Now you can import file.py
import pdo_functions  # assuming file.py contains functions/classes

import importlib
importlib.reload(pdo_functions)

In [ ]:
# Open SST data
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

In [ ]:
# Fill SST
sst_fill = xr.where(sst < -1.8, -1.8, sst)

In [ ]:
# Open convective weather environment data
cape  = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/wrf_cape/era5_daily_max_mucape_*s.nc', combine='nested', concat_dim='year')['__xarray_dataarray_variable__']
s06   = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/era5_s06_*s.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']
ndsev = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ndsev/diff_trapp_ndsev_mam_1940-2024.nc')['__xarray_dataarray_variable__']

In [ ]:
# Subset conus
s06_conus = s06.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

In [ ]:
# Change the shape of CAPE data
cape_stack = cape.stack(time=('year', 'date'))

# Convert time to datetime
year = cape_stack['time'].get_index('time').get_level_values('year')
date = cape_stack['time'].get_index('time').get_level_values('date')

datetime = pd.to_datetime(year.astype(str) + '-' + date.astype(str))

cape_stack = cape_stack.assign_coords(time=datetime)

In [ ]:
# Select 1940-2024
cape_1940_2024 = cape_stack.sel(time=cape_stack.time.dt.year.isin(np.arange(1940, 2025, 1)))
s06_1940_2024  = s06_conus.sel(time=s06_conus.time.dt.year.isin(np.arange(1940, 2025, 1)))

# Select March, April and May
cape_1940_2024_mam = cape_1940_2024.sel(time=cape_1940_2024.time.dt.month.isin([3, 4, 5])).dropna(dim='time')
s06_1940_2024_mam = s06_1940_2024.sel(time=s06_1940_2024.time.dt.month.isin([3, 4, 5])).dropna(dim='time')
ndsev_1940_2024_mam = ndsev

In [ ]:
# Take MAM mean for CAPE and S06 
cape_1940_2024_mam_mean = cape_1940_2024_mam.resample(time='YE').mean().load()
s06_1940_2024_mam_mean = s06_1940_2024_mam.resample(time='YE').mean().load()

In [ ]:
# Compute NDSEV during MAM
ndsev_monthly = np.zeros((85, 101, 237))
for i in range(85):
    ndsev_monthly[i] = ndsev_1940_2024_mam.isel(time=slice(i*92, (i+1)*92)).transpose('time', 'latitude', 'longitude').values.sum(axis=0)

In [ ]:
# Convert ndsev to xr
ndsev_1940_2024_mam_count = xr.DataArray(ndsev_monthly.swapaxes(0, 2).swapaxes(0, 1), coords={'latitude':ndsev['latitude'], 'longitude':ndsev['longitude'], 'time':cape_1940_2024_mam_mean['time']}, \
                                         dims=['latitude', 'longitude', 'time'])

In [ ]:
# Subset the eastern US
cape_east = cape_1940_2024_mam_mean.sel(longitude=slice(pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82))).sel(latitude=slice(47, 30))
s06_east = s06_1940_2024_mam_mean.sel(longitude=slice(pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82))).sel(latitude=slice(47, 30))
ndsev_east = ndsev_1940_2024_mam_count.sel(longitude=slice(pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82))).sel(latitude=slice(47, 30))

In [ ]:
# Land mask
land_mask = regionmask.defined_regions.natural_earth_v4_1_0.land_110.mask(cape_east)

east_land = xr.where(land_mask == 0, 1, np.nan)

cape_east_land = cape_east*east_land

In [ ]:
# Area averaging
weights = np.cos(np.deg2rad(cape_east.latitude))

cape_east_mean = cape_east.weighted(weights).mean(dim=('latitude', 'longitude'))
s06_east_mean = s06_east.weighted(weights).mean(dim=('latitude', 'longitude'))
ndsev_east_mean = ndsev_east.weighted(weights).mean(dim=('latitude', 'longitude'))

In [ ]:
# Detrend
cape_east_mean_de = pdo_functions.detrend_dim(cape_east_mean, 'time', 2)
s06_east_mean_de = pdo_functions.detrend_dim(s06_east_mean, 'time', 2)
ndsev_east_mean_de = pdo_functions.detrend_dim(ndsev_east_mean, 'time', 2)

In [ ]:
# Subset 1940-2024 SST
sst_1940_2024 = sst_fill.sel(time=sst_fill.time.dt.year.isin(np.arange(1940, 2025, 1)))
sst_1940_2024_mam = sst_1940_2024.sel(time=sst_1940_2024.time.dt.month.isin([3, 4, 5]))

# Take the seasonal mean
sst_1940_2024_mam_mean = sst_1940_2024_mam.resample(time='YE').mean()

In [ ]:
# Detrend SSTs
sst_de = pdo_functions.detrend_dim(sst_1940_2024_mam_mean, 'time', 2)

In [ ]:
# standardize everything
cape_east_mean_de_std = (cape_east_mean_de - cape_east_mean_de.mean(dim='time')) / cape_east_mean_de.std(dim='time')
s06_east_mean_de_std = (s06_east_mean_de - s06_east_mean_de.mean(dim='time')) / s06_east_mean_de.std(dim='time')
ndsev_east_mean_de_std = (ndsev_east_mean_de - ndsev_east_mean_de.mean(dim='time')) / ndsev_east_mean_de.std(dim='time')

sst_de_std = (sst_de - sst_de.mean(dim='time')) / sst_de.std(dim='time')

In [ ]:
# Correlate SSTs with convective weather environments
cape_corr = xr.corr(sst_de_std, cape_east_mean_de_std, dim='time')
s06_corr = xr.corr(sst_de_std, s06_east_mean_de_std, dim='time')
ndsev_corr = xr.corr(sst_de_std, ndsev_east_mean_de_std, dim='time')

In [ ]:
# Select domain
cape_corr_sel = cape_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(115, 180))
cape_corr_sel1 = cape_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(-180, -65))

s06_corr_sel = s06_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(115, 180))
s06_corr_sel1 = s06_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(-180, -65))

ndsev_corr_sel = ndsev_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(115, 180))
ndsev_corr_sel1 = ndsev_corr.sel(latitude=slice(70, -70)).sel(longitude=slice(-180, -65))

In [ ]:
fig, ax = plt.subplots(1, 3, subplot_kw=dict(projection=ccrs.Robinson(central_longitude=180)))

bounds = [-0.5, -0.45, -0.4, -0.35, -0.3, -0.25, -0.2, -0.15, -0.1, -0.05, -0.01, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

norm = c.BoundaryNorm(bounds, plt.get_cmap('cmr.fusion_r').N)

# SST lon/lat bounds
lons = cape_corr_sel['longitude'].values
lats = cape_corr_sel['latitude'].values
lon_min, lon_max = lons.min(), lons.max()
lat_min, lat_max = lats.min(), lats.max()

lons1 = cape_corr_sel1['longitude'].values
lats1 = cape_corr_sel1['latitude'].values
lon_min1, lon_max1 = lons1.min(), lons1.max()
lat_min1, lat_max1 = lats1.min(), lats1.max()

# Create bounding box of SST region
sst_bbox = box(lon_min, lat_min, lon_max, lat_max)
sst_bbox1 = box(lon_min1, lat_min1, lon_max1, lat_max1)

# Load land geometries from Natural Earth
land_shp = cfeature.NaturalEarthFeature('physical', 'land', '110m')
land_geoms = list(land_shp.geometries())

# Clip land geometries to SST bounding box
clipped_land = [geom.intersection(sst_bbox) for geom in land_geoms]
clipped_land1 = [geom.intersection(sst_bbox1) for geom in land_geoms]
# Remove empty geometries
clipped_land = [geom for geom in clipped_land if not geom.is_empty]
clipped_land1 = [geom for geom in clipped_land1 if not geom.is_empty]

# Create a ShapelyFeature for plotting
clipped_land_feature = ShapelyFeature(clipped_land, ccrs.PlateCarree(),
                                       facecolor='xkcd:light gray')
clipped_land_feature1 = ShapelyFeature(clipped_land1, ccrs.PlateCarree(),
                                       facecolor='xkcd:light gray')

# Get coastline geometries
coastline_shp = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
coast_geoms = list(coastline_shp.geometries())

# Clip coastlines to SST region
clipped_coast = [geom.intersection(sst_bbox) for geom in coast_geoms]
clipped_coast = [geom for geom in clipped_coast if not geom.is_empty]

clipped_coast1 = [geom.intersection(sst_bbox1) for geom in coast_geoms]
clipped_coast1 = [geom for geom in clipped_coast1 if not geom.is_empty]

# Plot clipped coastlines
clipped_coast_feature = ShapelyFeature(clipped_coast, ccrs.PlateCarree(),
                                       edgecolor='xkcd:gunmetal', facecolor='none', lw=0.25)

clipped_coast_feature1 = ShapelyFeature(clipped_coast1, ccrs.PlateCarree(),
                                       edgecolor='xkcd:gunmetal', facecolor='none', lw=0.25)

for i in range(3):
    # ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.)
    ax[i].add_feature(clipped_land_feature)
    ax[i].add_feature(clipped_land_feature1)
    ax[i].add_feature(clipped_coast_feature)
    ax[i].add_feature(clipped_coast_feature1)


ax[0].pcolormesh(cape_corr_sel['longitude'], cape_corr_sel['latitude'], cape_corr_sel, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
ax[0].pcolormesh(cape_corr_sel1['longitude'], cape_corr_sel1['latitude'], cape_corr_sel1, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

ax[1].pcolormesh(cape_corr_sel['longitude'], cape_corr_sel['latitude'], s06_corr_sel, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
ax[1].pcolormesh(cape_corr_sel1['longitude'], cape_corr_sel1['latitude'], s06_corr_sel1, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

ax[2].pcolormesh(cape_corr_sel['longitude'], cape_corr_sel['latitude'], ndsev_corr_sel, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
ax[2].pcolormesh(cape_corr_sel1['longitude'], cape_corr_sel1['latitude'], ndsev_corr_sel1, transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

ax[0].set_title('CAPE', pad=3)
ax[1].set_title('S06', pad=3)
ax[2].set_title('NDSEV', pad=3)

ax[0].set_title('(a)', loc='left', pad=3, x=0.1)
ax[1].set_title('(b)', loc='left', pad=3, x=0.1)
ax[2].set_title('(c)', loc='left', pad=3, x=0.1)

lat_min, lat_max = 30, 47
lon_min, lon_max = pdo_functions.convert_longitudes(-105), pdo_functions.convert_longitudes(-82)
rect_lons = [lon_min, lon_max, lon_max, lon_min, lon_min]
rect_lats = [lat_min, lat_min, lat_max, lat_max, lat_min]
ax[0].plot(rect_lons, rect_lats, color='black', lw=1, transform=ccrs.PlateCarree())

cax = plt.axes([0.1, .22, 0.8, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', \
                    ticks=[-0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5])
cbar.set_label(r'r', size=10, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=10, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.25)
cbar.outline.set_color('xkcd:gunmetal')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.6, 
                    wspace=0.05, 
                    hspace=0.05)